In [1]:
from agm_te.dataset import DataSet
from agm_te.model import train_agms, init_agms_from_loaders, _train_agm
import numpy as np
from te_datasim.lineargaussian import BVLinearGaussianSimulator

In [2]:
import torch; torch.set_printoptions(sci_mode=None)
# Check if CUDA is available
if torch.cuda.is_available():
    compute_device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    compute_device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


### Initialize the Simulator

In [3]:
bivar = BVLinearGaussianSimulator()
print("analytical TE X -> Y: ", bivar.analytic_transfer_entropy('X', 'Y'))
print("analytical TE Y -> X: ", bivar.analytic_transfer_entropy('Y', 'X'))

bivar_data_dict = {'X':[], 'Y':[]}
for i in range(10):
    X, Y = bivar.simulate(2000, seed=i)
    bivar_data_dict['X'].append(X)
    bivar_data_dict['Y'].append(Y)

bivar_data = DataSet(bivar_data_dict)

analytical TE X -> Y:  0.0
analytical TE Y -> X:  0.1276


### Estimate $T_{X \to Y}$

In [9]:
dataloader_1, dataloader_2 = bivar_data.get_TE_dataloaders(var_from='X', var_to='Y')
model_1, model_2 = init_agms_from_loaders(dataloader_1, dataloader_2, hidden_size=2, rnn_type='RNN')
model_1.to(compute_device)
model_2.to(compute_device)
model_1, model_2, loss = train_agms(model_1, dataloader_1, model_2, dataloader_2, batch_size=1, epochs=1000, learning_rate=1e-2, optimize='sgd')

Loss 1: 0.604406, Loss 2: 0.604452, Est. TE: 0.0


### Estimate $T_{Y \to X}$

In [10]:
dataloader_1, dataloader_2 = bivar_data.get_TE_dataloaders(var_from='Y', var_to='X')
model_1, model_2 = init_agms_from_loaders(dataloader_1, dataloader_2, hidden_size=2, rnn_type='RNN')
model_1.to(compute_device)
model_2.to(compute_device)
model_1, model_2, loss = train_agms(model_1, dataloader_1, model_2, dataloader_2, batch_size=1, epochs=1000, learning_rate=1e-2, optimize='sgd')

Loss 1: 0.739814, Loss 2: 0.61351, Est. TE: 0.1263
